### 图的实现<br>
- 以有向图为例：

In [48]:
class Node:
    
    def __init__(self,name):
        self.name = name 
    def getName(self):
        return self.name
    def __str__(self):
        return self.name
    def __len__(self):
        return len(1)

class Edge:
    
    def __init__(self,src,dest):
        self.src = src 
        self.dest = dest
    
    def getSource(self):
        return self.src
    
    def getDestination(self):
        return self.dest
    def __str__(self):
        return self.src.getName() + '->' + self.dest.getName()

class WeightedEdge(Edge):
    def __init__(self, src, dest, weight = 1.0):
        self.src = src
        self.dest = dest
        self.weight = weight
    def getWeight(self):
        return self.weight
    def __str__(self):
        return self.src.getName() + '->(' + str(self.weight) + ')'+ self.dest.getName()

以上是我们定义Graph的具体实现，首先图必定有的两个属性是node和edge，因此我们需要在创建实例的时候就初始化好这两个属性。 其次，我们可以根据需求定义一些方法，如：<br>
- 添加node<br><br>
- 添加边<br><br>
- 与当前node相关联的其它Nodes<br><br>
- 判断一个node是否在图中

In [49]:
class Digraph:
    
    def __init__(self):
        self.nodes = []
        self.edges = {}
    def addNode(self,node):
        if node in self.nodes:
            raise ValueError('Duplicate node!')
        else:
            self.nodes.append(node)
            self.edges[node] = []
    def addEdge(self,edge):
        src = edge.getSource()
        dest = edge.getDestination()
        if (src in self.nodes and dest in self.nodes):
            self.edges[src].append(dest)
        else:
            raise ValueError('Node not in graph')
    def childrenOf(self,node):
        return self.edges[node]
    def hasNode(self,node):
        return node in self.nodes
    def __str__(self):
        result = ''
        for src in self.nodes:
            for dest in self.edges[src]:
                result = result + src.getName() + '->'+ dest.getName() + '\n'
        return result[:-1] 

In [50]:
class Graph(Digraph):
    def addEdge(self, edge):
        Digraph.addEdge(self, edge)
        rev = Edge(edge.getDestination(), edge.getSource())
        Digraph.addEdge(self, rev)

- 我们通常使用邻接表的数据结构来表示Graph。Digraph类有两个实例变量，变量nodes是一个Python列表，其中的元素是Digraph中节点的名称。节点之间的连接是使用字典形式的邻接表来表示的。变量edges是一个字典，将Digraph中的每个Node对象映射到一个列表，其中元素是Node的子节点。<br><br>

- Graph类是Digraph的子类。除覆盖了addEdge方法以外，它继承了Digraph类的所有方法。那么为什么使用Graph作为Digraph的子类，而不是反过来呢？因为无向图的两点互相指向对方，可以看成是有向图的特殊情况。

### 图的搜索与遍历<br>
- 图的搜索指的就是从图的某一顶点开始，通过边到达不同的顶点，最终找到目标顶点的过程。搜索过程如果是整张图，那么就等于是图的遍历。根据搜索的顺序不同，图的搜索算法可分为“广度优先搜索”和“深度优先搜索”这两种。

#### 广度优先搜索 Breadth First Search (BFS)<br>
- 从图中某顶点v出发，在访问了v之后，依次访问v的各个未曾被访问过的邻接点，然后分别从这些邻接点出发依次访问它们的邻接点，并使得：先被访问的顶点的邻接点，先被访问，后被访问的顶点的邻接点，后被访问。直至图中所有已被访问的顶点的邻接点都被访问到。如果此时图中尚有顶点未被访问，则需要另选一个未曾被访问过的顶点作为新的起始点，重复上述过程，直至图中所有顶点都被访问到为止。<br><br>
- 每个被访问的顶点的邻接点，需要按被访问的顶点的访问顺序去放入特定的数据结构中，因此，你才能按添加顺序进行检查。有一个可实现这种目的的数据结构，那就是**队列(queue)**。

In [1]:
# deque相当于可以左右都能pop的list
from collections import deque

In [51]:
# 先定义一个打印路径的函数
def printPath(path):
    result = ""
    for i in range(len(path)):
        result = result +str(path[i])
        if i != len(path) - 1:
            result =  result + '->'
    return result

In [68]:
def BFS(graph,start,end):
    # 首先要有一整个图，和搜索的起止点
    initPath = deque([start])
    pathQueue = deque([initPath])
    while len(pathQueue) != 0:
        temPath = pathQueue.popleft()
        print('Current BFS path: ', printPath(temPath))
        lastNode = temPath[-1]
        if lastNode == end:
            return temPath
        for nextNode in graph.childrenOf(lastNode):
            if nextNode not in temPath:
                newPath = temPath + deque([nextNode])
                pathQueue.append(newPath)
    return None

In [69]:
def testSP():
    nodes = []
    for name in range(6):
        nodes.append(Node(str(name)))
    g = Digraph()
    for n in nodes:
        g.addNode(n)
    g.addEdge(Edge(nodes[0],nodes[1]))
    g.addEdge(Edge(nodes[0],nodes[2]))
    g.addEdge(Edge(nodes[1],nodes[3]))
    g.addEdge(Edge(nodes[2],nodes[3]))
    g.addEdge(Edge(nodes[3],nodes[4]))
    g.addEdge(Edge(nodes[3],nodes[5]))
    g.addEdge(Edge(nodes[1],nodes[5]))
    g.addEdge(Edge(nodes[5],nodes[1]))
    g.addEdge(Edge(nodes[4],nodes[2]))
    g.addEdge(Edge(nodes[5],nodes[4]))
    sp = BFS(g,nodes[5],nodes[2])
    print(printPath(sp))

In [70]:
testSP()

Current BFS path:  5
Current BFS path:  5->1
Current BFS path:  5->4
Current BFS path:  5->1->3
Current BFS path:  5->4->2
5->4->2


#### 深度优先搜索 Depth First Search (DFS)<br>
- 一般地，深度优先搜索算法开始时，会先选择起始节点的一个子节点，然后再选择这个子节点的一个子节点，以此类推，直到到达目标节点或者一个没有子节点的节点。然后，搜索开始回溯，返回到最近一个没有访问过的带有子节点的节点。遍历所有路径之后，算法就可以选择一个从起点到终点的最短路径（如果有）

In [72]:
def DFS(graph,start,end,path,shortest):
    path = path + [start]
    print('Current Path: ', printPath(path))
    if start == end:
        return path
    for node in graph.childrenOf(start):
        if node not in path:
            if shortest == None or len(path) < len(shortest):
                # 对初始节点的每一个子节点进行递归查找，查不到的会返回None，查得到的会保存
                newPath = DFS(graph,node,end,path,shortest)
                if newPath != None:
                    shortest = newPath
    return shortest

In [71]:
def testSP():
    nodes = []
    for name in range(5):
        nodes.append(Node(str(name)))
    g = Digraph()
    for n in nodes:
        g.addNode(n)
    g.addEdge(Edge(nodes[0],nodes[1]))
    g.addEdge(Edge(nodes[0],nodes[2]))
    g.addEdge(Edge(nodes[1],nodes[3]))
    g.addEdge(Edge(nodes[2],nodes[3]))
    g.addEdge(Edge(nodes[3],nodes[4]))
    sp = DFS(g,nodes[0],nodes[4],[],None)
    print(printPath(sp))

In [73]:
testSP()

Current Path:  0
Current Path:  0->1
Current Path:  0->1->3
Current Path:  0->1->3->4
Current Path:  0->2
Current Path:  0->2->3
Current Path:  0->2->3->4
0->2->3->4


#### 非递归的DFS

In [96]:
def DFS_1(graph,start,end):
    stack = []
    visited = []
    stack.append(start)
    visited.append(start)
    while len(stack) > 0:
        print('Current Path: ', printPath(visited))
        if visited[-1] == end:
            return visited
        x = stack[-1]
        for node in graph.childrenOf(x):
            if node not in visited:
                stack.append(node)
                visited.append(node)
                break

        if stack[-1] == x:
            stack.pop()
    return visited

In [83]:
def testSP():
    nodes = []
    for name in range(5):
        nodes.append(Node(str(name)))
    g = Digraph()
    for n in nodes:
        g.addNode(n)
    g.addEdge(Edge(nodes[0],nodes[1]))
    g.addEdge(Edge(nodes[0],nodes[2]))
    g.addEdge(Edge(nodes[1],nodes[3]))
    g.addEdge(Edge(nodes[2],nodes[3]))
    g.addEdge(Edge(nodes[3],nodes[4]))
    sp = DFS_1(g,nodes[0],nodes[4])
    print(printPath(sp))

In [97]:
testSP()

Current Path:  0
Current Path:  0->1
Current Path:  0->1->3
Current Path:  0->1->3->4
0->1->3->4
